# Stock Market Question Answering Agent

- answer questions about individual stocks using (mostly) OpenBB functions
- uses the OpenAPI tools functionality
- define a set of functions
- when prompting, give OpenAI the set of functions and their descriptions in a specified format
- if OpenAI needs to run a function to respond to the prompt, it will provide the function signature and ask you to provide the return values before continuing
  

In [1]:
import os
import sys
import dotenv
import warnings
from datetime import datetime, timedelta
import re

import pandas as pd

import IPython
from IPython.display import HTML, Image, Markdown, display

import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
# use firefox because it updates less often, can disable updates
# recommend importing profile from Chrome for cookies, passwords
# looks less like a bot with more user cruft in the profile
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.service import Service

import base64
import requests
import json

import openbb
from openbb import obb
from openbb_core.app.model.obbject import OBBject

import openai
from openai import OpenAI
import tiktoken

import langchain
from langchain.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

import wikipedia

# paid API for edgar filings
import sec_api
from sec_api import QueryApi, ExtractorApi

# free API for edgar filing
from sec_downloader import Downloader
import sec_parser as sp

dotenv.load_dotenv()

from BB_agent_tool import BB_agent_tool
from ainb_llm import get_response, agent_query

import pdb

# turn off excessive warnings
warnings.filterwarnings('ignore')


In [2]:
print(f'pandas         {pd.__version__}')
print(f'obb            {obb.system.version}')
print(f'selenium       {selenium.__version__}')
print(f'openai         {openai.__version__}')
print(f'langchain      {langchain.__version__}')
print(f'wikipedia      {wikipedia.__version__}')


pandas         2.2.2
obb            4.1.7
selenium       4.20.0
openai         1.28.0
langchain      0.1.20
wikipedia      (1, 4, 0)


# Connect to OpenBB

In [3]:
obb

OpenBB Platform v4.1.7

Utilities:
    /account
    /user
    /system
    /coverage

Routers:
    /commodity
    /crypto
    /currency
    /derivatives
    /econometrics
    /economy
    /equity
    /etf
    /fixedincome
    /index
    /news
    /quantitative
    /regulators
    /technical

Extensions:
    - commodity@1.0.4
    - crypto@1.1.5
    - currency@1.1.5
    - derivatives@1.1.5
    - econometrics@1.1.5
    - economy@1.1.5
    - equity@1.1.5
    - etf@1.1.5
    - fixedincome@1.1.5
    - index@1.1.5
    - news@1.1.5
    - quantitative@1.1.5
    - regulators@1.1.5
    - technical@1.1.6

    - alpha_vantage@1.1.5
    - benzinga@1.1.5
    - biztoc@1.1.5
    - cboe@1.1.5
    - ecb@1.1.5
    - federal_reserve@1.1.5
    - finra@1.1.5
    - finviz@1.0.4
    - fmp@1.1.5
    - fred@1.1.5
    - government_us@1.1.5
    - intrinio@1.1.5
    - nasdaq@1.1.6
    - oecd@1.1.5
    - polygon@1.1.5
    - sec@1.1.5
    - seeking_alpha@1.1.5
    - stockgrid@1.1.5
    - tiingo@1.1.5
    - tmx@1.0.2
 

In [4]:
# login with email and password
obb.account.login(email=os.environ['OPENBB_USER'], password=os.environ['OPENBB_PW'], remember_me=True)

In [5]:
obb.user

UserSettings

id: 066457c7-19a0-73f7-8000-a028fb0b743d
profile: {'hub_session': {'username': 'drucev', 'email': 'drucev@hotmail.com', 'primary_usage': 'personal', 'user_uuid': 'c866b4d2-c09b-4b13-abb7-a93f1ac3c2b7', 'token_type': 'bearer', 'access_token': SecretStr('**********')}}
credentials: {'nasdaq_api_key': None, 'biztoc_api_key': SecretStr('**********'), 'fred_api_key': SecretStr('**********'), 'tradier_api_key': None, 'tradier_account_type': None, 'fmp_api_key': SecretStr('**********'), 'intrinio_api_key': None, 'polygon_api_key': SecretStr('**********'), 'tradingeconomics_api_key': None, 'alpha_vantage_api_key': SecretStr('**********'), 'benzinga_api_key': None, 'tiingo_token': SecretStr('**********')}
preferences: {'cache_directory': '/Users/drucev/OpenBBUserData/cache', 'chart_style': 'dark', 'data_directory': '/Users/drucev/OpenBBUserData', 'export_directory': '/Users/drucev/OpenBBUserData/exports', 'metadata': True, 'output_type': 'OBBject', 'plot_enable_pywry': True, 'plot

In [6]:
obb.account

/account
    login
    logout
    save
    refresh
    

In [7]:
# Change a credential - only need once, gets stored in openbb cloud
# obb.user.credentials.polygon_api_key = os.environ['POLYGON_API_KEY']
# obb.user.credentials.alpha_vantage_api_key = os.environ['ALPHAVANTAGE_API_KEY']
# obb.user.credentials.fred_api_key = os.environ['FRED_API_KEY']
# obb.user.credentials.tiingo_token = os.environ['TIINGO_API_KEY']
# obb.user.credentials.fmp_api_key = os.environ['FMP_API_KEY']
# obb.user.credentials.biztoc_api_key = os.environ['BIZTOC_API_KEY']

# Save account changes to the Hub
# obb.account.save()

# Refresh account with latest changes since login
# obb.account.refresh()

# Logout
# obb.account.logout()

In [8]:
obb.equity

/equity
    /calendar
    /compare
    /darkpool
    /discovery
    /estimates
    /fundamental
    market_snapshots
    /ownership
    /price
    profile
    screener
    search
    /shorts
    

In [9]:
obb.equity.search("Merck", provider="nasdaq").to_df().head(3)


,symbol,name,nasdaq_traded,exchange,etf,round_lot_size,test_issue,cqs_symbol,nasdaq_symbol,next_shares
0,MRK,"Merck & Company, Inc. Common Stock (new)",Y,N,N,100.0,N,MRK,MRK,N


In [10]:
symbol = "MRK"
company = "Merck"

In [11]:
obj = obb.equity.price.performance(symbol)
obj


OBBject

id: 066457c7-1e23-727e-8000-08250f47236b
results: [{'symbol': 'MRK', 'one_day': 0.0239, 'wtd': None, 'one_week': 0.0168, 'mt...
provider: finviz
warnings: None
chart: None
extra: {'metadata': {'arguments': {'provider_choices': {'provider': 'finviz'}, 'sta...

In [12]:
obj.results


[FinvizPricePerformanceData(symbol=MRK, one_day=0.0239, wtd=None, one_week=0.0168, mtd=None, one_month=0.0533, qtd=None, three_month=0.047, six_month=0.28550000000000003, ytd=0.20829999999999999, one_year=0.132, two_year=None, three_year=None, four_year=None, five_year=None, ten_year=None, max=None, volatility_week=0.0134, volatility_month=0.0149, price=131.73, volume=7224765.0, average_volume=8210000.000000001, relative_volume=0.88, analyst_recommendation=None, analyst_score=1.48)]

In [13]:
obj.dict()


{'id': '066457c7-1e23-727e-8000-08250f47236b',
 'results': [{'symbol': 'MRK',
   'one_day': 0.0239,
   'wtd': None,
   'one_week': 0.0168,
   'mtd': None,
   'one_month': 0.0533,
   'qtd': None,
   'three_month': 0.047,
   'six_month': 0.28550000000000003,
   'ytd': 0.20829999999999999,
   'one_year': 0.132,
   'two_year': None,
   'three_year': None,
   'four_year': None,
   'five_year': None,
   'ten_year': None,
   'max': None,
   'volatility_week': 0.0134,
   'volatility_month': 0.0149,
   'price': 131.73,
   'volume': 7224765.0,
   'average_volume': 8210000.000000001,
   'relative_volume': 0.88,
   'analyst_recommendation': None,
   'analyst_score': 1.48}],
 'provider': 'finviz',
 'warnings': None,
 'chart': None,
 'extra': {'metadata': {'arguments': {'provider_choices': {'provider': 'finviz'},
    'standard_params': {'symbol': 'MRK'},
    'extra_params': {}},
   'duration': 171973541,
   'route': '/equity/price/performance',
   'timestamp': datetime.datetime(2024, 5, 15, 23, 24, 

In [14]:
# use REST API on server running locally
# uvicorn openbb_core.api.rest_api:app --host 0.0.0.0 --port 8000 --reload
# REST API documentation - http://127.0.0.1:8000/docs
# openapi.json : http://127.0.0.1:8000/openapi.json

# not turning on authentication
# msg = "some_user:some_pass"
# msg_bytes = msg.encode('ascii')
# base64_bytes = base64.b64encode(msg_bytes)
# base64_msg = base64_bytes.decode('ascii')

url = f"http://127.0.0.1:8000/api/v1/equity/price/quote?provider=yfinance&symbol={symbol}"
# headers = {"accept": "application/json", "Authorization": f"Basic {base64_msg}"}
headers = {"accept": "application/json"}

response = requests.get(url=url, headers=headers)

response.json()


{'results': [{'symbol': 'MRK',
   'asset_type': 'EQUITY',
   'name': 'Merck & Co., Inc.',
   'exchange': 'NYQ',
   'bid': 131.73,
   'bid_size': 800,
   'ask': 131.68,
   'ask_size': 800,
   'last_price': 131.73,
   'open': 128.88,
   'high': 131.965,
   'low': 128.72,
   'volume': 7223237,
   'prev_close': 128.66,
   'year_high': 133.1,
   'year_low': 99.14,
   'ma_50d': 126.818,
   'ma_200d': 114.56235,
   'volume_average': 8261309.0,
   'volume_average_10d': 6897520.0,
   'currency': 'USD'}],
 'provider': 'yfinance',
 'warnings': None,
 'chart': None,
 'extra': {'metadata': {'arguments': {'provider_choices': {'provider': 'yfinance'},
    'standard_params': {'symbol': 'MRK'},
    'extra_params': {'use_cache': True, 'source': 'iex'}},
   'duration': 175450917,
   'route': '/equity/price/quote',
   'timestamp': '2024-05-15T23:24:33.905377'}}}

In [15]:
data = obb.equity.price.historical(symbol, provider="polygon")
data.to_dataframe()


,open,high,low,close,volume,vwap,transactions
date,,,,,,,
2023-05-15,117.14,117.740,115.4900,116.37,5697236.0,116.3517,70438
2023-05-16,115.88,116.760,115.2900,116.08,4278281.0,116.2071,60730
2023-05-17,116.37,116.655,113.4800,114.76,7210383.0,114.5498,78904
2023-05-18,114.14,115.070,113.3300,114.00,6437144.0,114.0864,75952
2023-05-19,114.33,116.240,114.1000,115.49,7647080.0,115.4905,63052
...,...,...,...,...,...,...,...
2024-05-08,130.58,131.510,129.3300,129.55,6580073.0,129.7913,74021
2024-05-09,128.94,130.500,128.9400,130.23,9038061.0,129.9542,70936
2024-05-10,130.82,130.880,129.9500,130.06,5540149.0,130.3223,73151


In [16]:
# use the local rest server
data = []
symbol2="SPY"
url = f"http://127.0.0.1:8000/api/v1/equity/price/historical?provider=polygon&symbol={symbol2}"
headers = {"accept": "application/json"}

response = requests.get(url, headers=headers, timeout=3)

if response.status_code == 200:
  data = OBBject.model_validate(response.json())

data.to_df()


,close,high,low,open,transactions,volume,vwap
date,,,,,,,
2023-05-15,413.01,413.4300,410.230,412.22,427865,54289383.0,412.1201
2023-05-16,410.25,412.8150,410.240,411.86,447536,57705495.0,411.3774
2023-05-17,415.23,415.8550,410.635,412.35,585038,86786957.0,413.4302
2023-05-18,419.23,419.6700,414.670,414.90,672492,97177195.0,417.1828
2023-05-19,418.62,420.7200,417.350,420.17,642871,103793317.0,419.0232
...,...,...,...,...,...,...,...
2024-05-08,517.19,517.7400,515.140,515.26,369150,42012599.0,516.9211
2024-05-09,520.17,520.2074,516.705,517.38,399512,43583253.0,519.1084
2024-05-10,520.84,522.6350,519.590,521.81,428536,52201942.0,520.8834


In [17]:
results = obb.equity.search(query='marvell', is_symbol=False, provider='nasdaq', use_cache=True)
[(r.symbol, r.name) for r in results.results]


[('MRVL', 'Marvell Technology, Inc. - Common Stock')]

In [18]:
# multiple symbols
quotes = obb.equity.price.quote("td,schw,jpm,ms", provider="fmp")
quotes.to_df()


,symbol,name,exchange,last_price,last_timestamp,open,high,low,volume,prev_close,...,year_high,year_low,price_avg50,price_avg200,avg_volume,market_cap,shares_outstanding,eps,pe,earnings_announcement
0,TD,The Toronto-Dominion Bank,NYSE,56.89,2024-05-15 20:00:02+00:00,56.90,57.12,56.52,1821579,56.72,...,66.15,54.12,58.6438,60.15885,3059839,1.008148e+11,1772100000,4.61,12.34,2024-05-23 12:00:00+00:00
1,SCHW,The Charles Schwab Corporation,NYSE,78.68,2024-05-15 20:00:02+00:00,77.42,78.79,77.38,6369948,77.07,...,78.79,48.66,72.0174,63.23660,7058424,1.398364e+11,1777280000,2.39,32.92,2024-07-16 04:00:00+00:00
2,JPM,JPMorgan Chase & Co.,NYSE,202.11,2024-05-15 20:00:02+00:00,202.01,202.69,199.77,8115081,201.51,...,202.69,134.40,192.7314,166.44000,8847162,5.803932e+11,2871670000,16.56,12.20,2024-07-12 00:00:00+00:00
3,MS,Morgan Stanley,NYSE,100.52,2024-05-15 20:00:47+00:00,99.00,100.98,99.00,6918368,99.38,...,100.98,69.42,91.6292,85.79915,7980975,1.633611e+11,1625160000,5.50,18.28,2024-07-16 04:00:00+00:00


In [19]:
# multiple providers

df = pd.DataFrame()

df["yfinance"] = (
  obb.equity.fundamental.balance(symbol, provider="yfinance", limit=3)
  .to_df().get("total_assets")
)

df["fmp"] = (
  obb.equity.fundamental.balance(symbol, provider="fmp", limit=3)
  .to_df().get("total_assets")
)

df["polygon"] = (
  obb.equity.fundamental.balance(symbol, provider="polygon", limit=3)
  .to_df().get("total_assets")
)

df

,yfinance,fmp,polygon
0,1.066750e+11,1.066750e+11,1.066750e+11
1,1.091600e+11,1.091600e+11,1.091600e+11
2,1.056940e+11,1.056940e+11,1.056940e+11
3,9.158800e+10,NaN,NaN


In [20]:
obb.news.company(symbol, provider='polygon', limit=5).to_df()


,title,text,images,url,symbols,source,id,amp_url,publisher
date,,,,,,,,,
2024-05-13 15:16:00+00:00,Bristol Myers (BMY) Fails to Meet Goal in Opdi...,Bristol Myers' (BMY) late-stage label-expandin...,[{'url': 'https://staticx-tuner.zacks.com/imag...,https://www.zacks.com/stock/news/2272954/brist...,"AZN,BMY,MRK,LGND",Zacks Equity Research,DZ7EFMYgND4Mcq1fh_0B9LZ6EWP-9-JQz_pR6koaUmA,https://www.zacks.com/amp/stock/news/2272954/b...,{'favicon_url': 'https://s3.polygon.io/public/...
2024-05-14 13:25:00+00:00,Merck (MRK) Ends Keytruda Combo Melanoma Study...,Merck (MRK) discontinues the Keytruda plus vib...,[{'url': 'https://staticx-tuner.zacks.com/imag...,https://www.zacks.com/stock/news/2273402/merck...,"MRK,MRNA,LGND,ANIP",Zacks Equity Research,R1VrrdEUIYHeJqiOWSzIxRUOlqB_Oa1pGnAYJYBm06w,https://www.zacks.com/amp/stock/news/2273402/m...,{'favicon_url': 'https://s3.polygon.io/public/...
2024-05-15 05:22:49+00:00,MannKind Outperforms As Tyvaso DPI Captures Ma...,MannKind's long-term liabilities exceed its as...,[{'url': 'https://static.seekingalpha.com/cdn/...,https://seekingalpha.com/article/4693345-mannk...,"LQDA,MRK,UTHR,MNKD",Stephen Ayers,fQ4xeEwbTVYO0LoI2YgpHdzGmeDLS3cKjfUdvyuUBfA,None,{'favicon_url': 'https://s3.polygon.io/public/...
2024-05-15 12:45:36+00:00,Pfizer: Best Trading Momentum Since 2021 And 6...,Pfizer has been a losing investment for severa...,[{'url': 'https://static.seekingalpha.com/cdn/...,https://seekingalpha.com/article/4693405-pfize...,"ABBV,AMGN,AZN,AZNCF,BMY,BMYMP,GILD,JNJ,LLY,MRK...",Paul Franke,q9bmkpb18oxPdfIlrtS-uA0trrZ-yMQWBaO6f28ZfkM,None,{'favicon_url': 'https://s3.polygon.io/public/...
2024-05-15 17:11:00+00:00,"Top Stock Reports for Broadcom, Merck & Airbnb",Today's Research Daily features new research r...,[{'url': 'https://staticx-tuner.zacks.com/imag...,https://www.zacks.com/research-daily/2274193/t...,"PCG,MRK,OKE,ENB,AVGO,ABNB",Sheraz Mian,m4MxY768tHMwb6pp0h-AmvZU_TduiMvnjVNHpoVNjDQ,None,{'favicon_url': 'https://s3.polygon.io/public/...


# Prompt OpenAI 

In [21]:
MODEL = "gpt-4o"

# MAX_INPUT_TOKENS = 65536     
MAX_OUTPUT_TOKENS = 4096    # max in current model
MAX_RETRIES = 3
TEMPERATURE = 0



In [22]:
messages = [{"role": "user", "content": "what is the airspeed velocity of an unladen swallow?"}]
response = get_response(messages, tools=[])
response_str = response.choices[0].message.content
response_str = response_str.replace("$", "\\\$")

print(response_str)

The airspeed velocity of an unladen swallow is often a topic of curiosity and humor thanks to its reference in the movie "Monty Python and the Holy Grail." However, if we are to address this question scientifically, we need to specify the type of swallow.

For the European Swallow (Hirundo rustica), a commonly quoted figure is around 11 meters per second or approximately 24 miles per hour. This estimation is based on studies and calculations considering the bird's typical wing beat frequency and flight dynamics.

It's important to note that various factors can influence an individual swallow's speed, such as wind conditions, altitude, and whether the bird is involved in specific behaviors like foraging or migration.


# Question Answering Agent

In [23]:
# make a bespoke tool 

def get_10k_item1_from_symbol(symbol):
    """
    Get item 1 of the latest 10-K annual report filing for a given symbol.

    Args:
        symbol (str): The symbol of the equity.

    Returns:
        str: The item 1 of the latest 10-K annual report filing, or None if not found.

    """
    item1_text = None
    try:
        # sec needs you to identify yourself for rate limiting
        dl = Downloader(os.getenv("SEC_FIRM"), os.getenv("SEC_USER"))
        html = dl.get_filing_html(ticker=symbol, form="10-K")
        elements: list = sp.Edgar10QParser().parse(html)
        tree = sp.TreeBuilder().build(elements)
        sections = [n for n in tree.nodes if n.text.startswith("Item")]
        item1_node = sections[0]
        item1_text = "\n".join([n.text for n in sections[0].get_descendants()])
    except:
        return None
    # always return a list of dicts
    return [{'item1': item1_text}]

fn_metadata = {
    "name": "get_10k_item1_from_symbol",
    "description": "Given a stock symbol, gets item 1 of the company's latest 10-K annual report filing.",
    "openapi_path" : None,
    "callable": get_10k_item1_from_symbol,
    "parameters": {
        "symbol": {
            "type": "string",
            "description": "The symbol to get the 10-K item 1 for"
            }
        },
    "example_parameter_values": [{
        "symbol": "MSFT",
    }],
}

tool = BB_agent_tool(**fn_metadata)
tool_response = json.loads(tool(symbol="MSFT"))
len(tool_response[0]['item1'])

# create tool dict based on metadata
# def make_tool_dict(fn_metadata):
#     retdict  = {
#         "type": "function",
#         "function": {
#             "name": fn_metadata["name"],
#             "description": fn_metadata["description"],
#             "parameters": {
#                 "type": "object",
#                 "properties": fn_metadata["parameters"],
#                 "required": [k for k in fn_metadata["parameters"].keys()]
#                 }
#             }
#         }
#     return retdict

# tools[fn_metadata["name"]] = fn_metadata
# tools[fn_metadata["name"]]["tooldict"] = make_tool_dict(fn_metadata)


143226

In [24]:
user_message = "Please summarize item 1 from the latest MSFT annual report"
agent_query(user_message, verbose=False)


get_10k_item1_from_symbol({'symbol': 'MSFT'}) -> [{"item1": "Note About Forward-Looking Statements\nThis report includes estimates, projections, stat...


### Summary of Item 1 from Microsoft's Latest 10-K Annual Report

**Note About Forward-Looking Statements:** This section includes statements about Microsoft's business plans, objectives, and expected performance. Such forward-looking statements might include words like "believe," "expect," "anticipate," and "estimate," and they are subject to risks that might cause actual results to differ.

**Business Description:**
- **Mission:** Empower every person and organization on the planet to achieve more by creating opportunities and fostering growth globally.
- **Core Areas:**
  - **AI and Productivity Platform:** Develops platforms and tools powered by AI to support business competitiveness and improve educational, health, and public-sector efficiency.
  - **Microsoft Cloud:** Integrates various technologies to unlock digital spend value, innovate with AI, and simplify security and management.

**Product Offerings:**
- **Software & Services:** Includes operating systems, productivity tools (Office 365), business solutions, collaboration apps (Teams), and development tools.
- **Devices:** PCs, tablets, gaming consoles, and accessories.
- **Customer Solutions:** Cloud-based solutions for software, services, content, and consulting.

**Strategic Ambitions:**
1. **Reinvent Productivity and Business Processes**: Innovative tools like Microsoft 365, Teams, and Dynamics 365 aim to enhance business operations and productivity using AI.
2. **Build Intelligent Cloud and Intelligent Edge**: Investment in cloud services (Azure), AI capabilities, and edge computing to drive enterprise digital transformation.
3. **Create More Personal Computing**: Innovations in devices and user interfaces, including Windows 11 and AI-powered solutions.

**Corporate Social Responsibility (CSR):**
- **Sustainability:** Commitment to being carbon negative, water positive, and zero-waste by 2030.
- **Racial Equity:** Focused on strengthening inclusion and increasing representation within the company and its ecosystem.
- **Investment in Skills:** Programs for digital skills training, cybersecurity workforce development, and AI literacy.

**Human Capital:**
- As of June 30, 2023, Microsoft employed approximately 221,000 people globally, focusing on creating a respectful and inclusive work environment.

**Operating Segments:**
- **Productivity and Business Processes:** Includes Office products, LinkedIn, and Dynamics.
- **Intelligent Cloud:** Comprises Azure, server products, and enterprise services.
- **More Personal Computing:** Encompasses Windows, devices, gaming, and search advertising.

**Research and Development:**
- Focused on emerging technologies, AI, IoT, and data platforms, with significant investments in cloud, security, and mixed reality solutions.

**Distribution and Sales:**
- Products are distributed via OEMs, direct sales, and a network of resellers, with a variety of licensing options for organizations of all sizes.

**Legal and Regulatory Risks:**
- Acknowledges potential challenges from competition laws, tax policies, cyber threats, and data protection regulations globally.

The section provides a comprehensive view of Microsoft's business model, strategic objectives, products, market positions, and the steps it is taking to address various risks and opportunities.

# Add more OpenBB tools
Map OpenBB functions to OpenAI tools

In [25]:
# TODO:

# move object to a file , move llm calls to a file

# make a streamlit app that maintains conversation

# use assistants API


In [26]:
# load the OpenAPI / swagger spec from
# http://127.0.0.1:8000/openapi.json
with open("openapi.json", 'r') as file:
    data = json.load(file)

str(data)[:2000]

"{'openapi': '3.1.0', 'info': {'title': 'OpenBB Platform API', 'description': 'This is the OpenBB Platform API.', 'termsOfService': 'http://example.com/terms/', 'contact': {'name': 'OpenBB Team', 'url': 'https://openbb.co/', 'email': 'hello@openbb.co'}, 'license': {'name': 'MIT', 'url': 'https://github.com/OpenBB-finance/OpenBBTerminal/blob/develop/LICENSE'}, 'version': '1'}, 'servers': [{'url': 'http://localhost:8000', 'description': 'Local OpenBB development server'}], 'paths': {'/api/v1/commodity/lbma_fixing': {'get': {'tags': ['commodity'], 'summary': 'Lbma Fixing', 'description': 'Daily LBMA Fixing Prices in USD/EUR/GBP.', 'operationId': 'commodity_lbma_fixing', 'parameters': [{'name': 'provider', 'in': 'query', 'required': False, 'schema': {'enum': ['nasdaq'], 'const': 'nasdaq', 'type': 'string', 'default': 'nasdaq', 'title': 'Provider'}}, {'name': 'asset', 'in': 'query', 'required': False, 'schema': {'enum': ['gold', 'silver'], 'type': 'string', 'description': 'The metal to get 

In [27]:
# list all the equity functions
for path_str, fn_json in data['paths'].items():
    if path_str.find('equity') != -1:
        print(path_str)


/api/v1/equity/calendar/ipo
/api/v1/equity/calendar/dividend
/api/v1/equity/calendar/splits
/api/v1/equity/calendar/earnings
/api/v1/equity/compare/peers
/api/v1/equity/compare/groups
/api/v1/equity/estimates/price_target
/api/v1/equity/estimates/historical
/api/v1/equity/estimates/consensus
/api/v1/equity/estimates/analyst_search
/api/v1/equity/estimates/forward_sales
/api/v1/equity/estimates/forward_eps
/api/v1/equity/darkpool/otc
/api/v1/equity/discovery/gainers
/api/v1/equity/discovery/losers
/api/v1/equity/discovery/active
/api/v1/equity/discovery/undervalued_large_caps
/api/v1/equity/discovery/undervalued_growth
/api/v1/equity/discovery/aggressive_small_caps
/api/v1/equity/discovery/growth_tech
/api/v1/equity/discovery/top_retail
/api/v1/equity/discovery/upcoming_release_days
/api/v1/equity/discovery/filings
/api/v1/equity/fundamental/multiples
/api/v1/equity/fundamental/balance
/api/v1/equity/fundamental/balance_growth
/api/v1/equity/fundamental/cash
/api/v1/equity/fundamental/r

In [28]:
data['paths']['/api/v1/equity/search']


{'get': {'tags': ['equity'],
  'summary': 'Search',
  'description': 'Search for stock symbol, CIK, LEI, or company name.',
  'operationId': 'equity_search',
  'parameters': [{'name': 'provider',
    'in': 'query',
    'required': True,
    'schema': {'enum': ['cboe', 'intrinio', 'nasdaq', 'sec', 'tmx', 'tradier'],
     'type': 'string',
     'title': 'Provider'}},
   {'name': 'query',
    'in': 'query',
    'required': False,
    'schema': {'type': 'string',
     'description': 'Search query.',
     'default': '',
     'title': 'Query'},
    'description': 'Search query.'},
   {'name': 'is_symbol',
    'in': 'query',
    'required': False,
    'schema': {'type': 'boolean',
     'description': 'Whether to search by ticker symbol.',
     'default': False,
     'title': 'Is Symbol'},
    'description': 'Whether to search by ticker symbol.'},
   {'name': 'use_cache',
    'in': 'query',
    'required': False,
    'schema': {'anyOf': [{'type': 'boolean'}, {'type': 'null'}],
     'descriptio

In [29]:
fn_metadata = {
    "name": "get_equity_search_symbol",
    "description": "Given a search string, get the stock symbol of the top company whose name best matches the search string.",
    "openapi_path" : '/api/v1/equity/search',
    "parameters": {
        "query": {
            "type": "string",
            "description": "The search string to match to the stock symbol."
            },""
        "limit": {
            "type": "integer",
            "description": "The number of results to return. Pick a small number from 1 to 10 and choose the best response."
            }
        },
    "example_parameter_values": [{
        "query": "Broadcom",
    }],
    # "singular": 1,
}

tool = BB_agent_tool(**fn_metadata)


In [30]:
BB_agent_tool.agent_registry['get_equity_search_symbol'](query="Southwest Air")

'[{"symbol": "LUV", "name": "SOUTHWEST AIRLS CO COM", "dpm_name": "Susquehanna Securities, LLC", "post_station": "5/1"}]'

In [31]:
agent_query("What is the stock symbol for Southwest airlines?", verbose=False)


get_equity_search_symbol({'query': 'Southwest Airlines', 'limit': 1}) -> []
get_equity_search_symbol({'query': 'Southwest', 'limit': 10}) -> [{"symbol": "CSWC", "name": "CAPITAL SOUTHWEST CORP COM", "dpm_name": "Wolverine Trading, LLC", "pos...


The stock symbol for Southwest Airlines is **LUV**.

In [32]:
agent_query("What is the stock symbol for yakimee air?", verbose=False)


get_equity_search_symbol({'query': 'yakimee air', 'limit': 1}) -> []


I couldn't find any stock symbol for "yakimee air." It appears there might have been a spelling error or the company name might not be correct. Could you please check the name and try again? If you need help with another company, feel free to ask!

In [33]:
# agent is able to first get symbol, then get annual report for symbol 
agent_query("What is item 1 from the 10k annual report for Microsfot?", verbose=True)


Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content='It looks like there’s a typo in "Microsfot." I believe you meant "Microsoft." Let me retrieve the relevant information for Microsoft.\n\nFirst, I\'ll confirm the ticker symbol for Microsoft, just to be certain.', role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_ssutAB8VZEHbWYfd33zdwchX', function=Function(arguments='{"query":"Microsoft","limit":1}', name='get_equity_search_symbol'), type='function')]))
[ChatCompletionMessageToolCall(id='call_ssutAB8VZEHbWYfd33zdwchX', function=Function(arguments='{"query":"Microsoft","limit":1}', name='get_equity_search_symbol'), type='function')]
get_equity_search_symbol({'query': 'Microsoft', 'limit': 1}) -> [{"symbol": "MSFT", "name": "MICROSOFT CORP COM", "dpm_name": "Susquehanna Securities, LLC", "post_s...
Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content='The symb

The **Item 1** from Microsoft's latest 10-K annual report provides an extensive overview of the company's business, including key aspects like their mission, product offerings, competitive landscape, and strategic focus areas. Here are some of the highlights:

### General Overview
- **Mission**: Microsoft's mission is to empower every person and every organization on the planet to achieve more.
- **Core Products and Services**: They offer a broad range of software, services, devices, and solutions that deliver unique value to customers. Key areas include cloud services, productivity tools, gaming, artificial intelligence, and more.

### Product Segments
1. **Productivity and Business Processes**:
    - Includes Office Commercial (Office 365, Microsoft Teams), Office Consumer, LinkedIn, and Dynamics business solutions.
    - Focuses on improving productivity, enhancing communication, and aiding business processes via cloud and on-premises software.

2. **Intelligent Cloud**:
    - Comprises server products and cloud services like Azure, SQL Server, Windows Server, and developer tools.
    - Azure AI helps organizations transform by bringing intelligence and insights.

3. **More Personal Computing**:
    - Encompasses Windows, devices (Surface, HoloLens), gaming (Xbox), and search/news advertising.
    - Focuses on delivering personalized computing experiences, gaming, and search services.

### Innovations and Strategic Focus
- **Artificial Intelligence (AI)**: AI is a major focus, integrated across various services to support better business and personal productivity.
- **Cloud Computing**: Investment in cloud services, particularly Azure, to drive digital transformation for businesses and provide scalable computing solutions.
- **Gaming**: Continued investment in gaming content, cloud gaming services, and hardware to enhance user engagement and growth across Xbox and related services.

### Other Key Points
- **Commitment to Security**: Microsoft emphasizes secure and reliable services to protect customer data and maintain service integrity.
- **Sustainability and Corporate Responsibility**: Efforts are ongoing to address environmental sustainability, such as becoming carbon negative and increasing water positivity.
- **Support for Diversity and Inclusion**: Investment initiatives aimed at enhancing representation and inclusion within Microsoft and beyond.

The above summary captures the essence of Microsoft's business and strategic direction as delineated in Item 1 of their latest 10-K report. For more detailed information, you may refer to the complete 10-K filing.

In [61]:
agent_query(f"What is the last market quote for symbol {symbol}?", verbose=False)


get_price_performance_json({'symbol': 'MRK'}) -> [{"symbol": "MRK", "one_day": 0.0239, "wtd": null, "one_week": 0.0168, "mtd": null, "one_month": 0.0...


The latest market quote for Merck & Co., Inc. (symbol: MRK) is as follows:

- **Price:** \\$131.73
- **Volume:** 7,224,765
- **Average Volume:** 8,210,000
- **Relative Volume:** 0.88
- **One Day Price Change:** +2.39%
- **One Week Price Change:** +1.68%
- **One Month Price Change:** +5.33%
- **Three Month Price Change:** +4.70%
- **Six Month Price Change:** +28.55%
- **Year to Date Price Change:** +20.83%
- **One Year Price Change:** +13.20%
- **Weekly Volatility:** 1.34%
- **Monthly Volatility:** 1.49%

If you need more information, feel free to ask!

In [35]:
agent_query(f"What is the last market quote for Consolidated Agglomerators", verbose=False)


get_equity_search_symbol({'query': 'Consolidated Agglomerators', 'limit': 5}) -> []


I couldn't find any stock symbol for "Consolidated Agglomerators." Please verify the company name or provide an alternative company name or query.

In [36]:
agent_query(f"What is the current weather in Las Vegas", verbose=False)


I am unable to provide information about the weather. You might want to check a weather website or app for the current conditions in Las Vegas.

In [37]:
tool = BB_agent_tool(
    name="get_company_profile_json",
    description="Given a stock symbol, get general background data about the company such as company name, industry, and sector data in JSON format",
    openapi_path='/api/v1/equity/profile',
    parameters={
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    example_parameter_values=[{
        "symbol": "NVDA",
    }],
)

agent_query(f"Can you provide a basic company profile of symbol {symbol}", verbose=False)


get_company_profile_json({'symbol': 'MRK'}) -> [{"symbol": "MRK", "name": "Merck & Co Inc", "cik": null, "cusip": null, "isin": null, "lei": null, ...


### Merck & Co., Inc. (Ticker: MRK)

**Industry:** Healthcare  
**Stock Exchange:** NYSE  
**Index Membership:** DJIA, S&P 500  
**Market Cap:** \\$333.65 Billion  
**Beta:** 0.41  
**Institutional Ownership:** 79.05%  
**Shares Outstanding:** 2.53 Billion  
**Floating Shares:** 2.53 Billion  
**Short Interest:** 18.75 Million  

**Business Description:**  
Merck & Co., Inc. is a global healthcare company that provides health solutions through its range of prescription medicines, vaccines, biologic therapies, animal health products, and consumer care products. The company operates through three primary segments:

- **Pharmaceutical:** This segment includes Merck's human health pharmaceutical and vaccine products.
- **Animal Health:** Focuses on discovering, developing, manufacturing, and marketing animal health products, such as pharmaceuticals and vaccines for the prevention, treatment, and control of disease in livestock and companion animals.
- **Other:** Comprises sales for the company’s non-reportable segments of healthcare services.

**Founding Year:** 1891  
**Headquarters:** Rahway, NJ, USA  
**Employees:** Approximately 72,000

If you need more specific information, feel free to ask!

In [38]:
tool = BB_agent_tool(
    name="get_equity_shorts_short_interest",
    description="Given a stock symbol, get data on short volume and days to cover in JSON format.",
    openapi_path='/api/v1/equity/shorts/short_interest',
    parameters={
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    example_parameter_values=[{
        "symbol": "NVDA",
    }],
)

agent_query(f"Provide latest statistics on short interest for symbol {symbol}", verbose=False)


get_equity_shorts_short_interest({'symbol': 'MRK'}) -> [{"settlement_date": "2021-07-15", "symbol": "MRK", "issue_name": "Merck & Co., Inc.", "market_class...


As of the latest available data, here are the short interest statistics for Merck & Co., Inc. (symbol: MRK):

- **Settlement Date:** April 30, 2024
- **Current Short Position:** 18,754,018 shares
- **Previous Short Position:** 20,429,972 shares
- **Average Daily Volume:** 7,551,651 shares
- **Days to Cover:** 2.48 days
- **Changes:** 
  - Change in Short Position: -1,675,954 shares
  - Change Percentage: -8.20%

Would you like more detailed historical data or additional information?

In [39]:
tool = BB_agent_tool(
    name="get_equity_fundamental_historical_splits",
    description="Given a stock symbol, get the company's historical stock splits in JSON format.",
    openapi_path='/api/v1/equity/fundamental/historical_splits',
    parameters={
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    example_parameter_values=[{
        "symbol": "NVDA",
    }],
)


In [40]:
agent_query(f"Provide historical split information for symbol AAPL", verbose=False)

get_equity_fundamental_historical_splits({'symbol': 'AAPL'}) -> [{"date": "2020-08-31", "numerator": 4.0, "denominator": 1.0, "split_ratio": null, "label": "August ...


Here is the historical stock split information for Apple Inc. (symbol: AAPL):

1. **August 31, 2020**: 4-for-1 split
2. **June 09, 2014**: 7-for-1 split
3. **February 28, 2005**: 2-for-1 split
4. **June 21, 2000**: 2-for-1 split
5. **June 16, 1987**: 2-for-1 split

If you have any other questions or need further information, feel free to ask!

In [41]:
tool = BB_agent_tool(
    name="get_balance_sheet_json",
    description="Given a stock symbol, get the latest balance sheet data with assets and liabilities for the company in JSON format.",
    openapi_path='/api/v1/equity/fundamental/balance',
    parameters={
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    example_parameter_values=[{
        "symbol": "NVDA",
    }],
    singular=1
)

agent_query(f"what are the latest total assets for symbol {symbol}", verbose=False)


get_balance_sheet_json({'symbol': 'MRK'}) -> {"period_ending": "2023-12-31", "fiscal_period": "FY", "fiscal_year": 2023, "filing_date": "2024-04-...


The latest total assets for Merck & Co., Inc. (symbol: MRK) as of the fiscal year ending December 31, 2023, are \\$106.675 billion USD. 

For further details, you can refer to their [official 10-K filing](https://www.sec.gov/Archives/edgar/data/310158/000119312524093194/d807955d10ka.htm).

In [42]:
# can give by year
# obb.equity.fundamental.balance(symbol='NVDA', period='annual', fiscal_year='2023', limit=1)
# also growth from prior period obb.equity.fundamental.balance_growth

In [43]:
tool = BB_agent_tool(
    name="get_cash_flow_json",
    description="Given a stock symbol, get the latest cash flow statement data for the company in JSON format.",
    openapi_path='/api/v1/equity/fundamental/cash',
    parameters={
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    example_parameter_values=[{
        "symbol": "NVDA",
    }],
    singular=1
)

agent_query(f"what was the most recent cash flow from operations for stock symbol {symbol}", verbose=False)


get_cash_flow_json({'symbol': 'MRK'}) -> {"period_ending": "2023-12-31", "fiscal_period": "FY", "fiscal_year": 2023, "filing_date": "2024-04-...


The most recent cash flow from operations for Merck & Co., Inc. (NYSE: MRK) for the fiscal year ending December 31, 2023, was \\$13,006,000,000.

For more details, you can view the full cash flow statement [here](https://www.sec.gov/Archives/edgar/data/310158/000119312524093194/d807955d10ka.htm).

In [44]:
tool.agent_registry

{'get_10k_item1_from_symbol': <BB_agent_tool.BB_agent_tool at 0x319356790>,
 'get_equity_search_symbol': <BB_agent_tool.BB_agent_tool at 0x319326810>,
 'get_company_profile_json': <BB_agent_tool.BB_agent_tool at 0x3195891d0>,
 'get_equity_shorts_short_interest': <BB_agent_tool.BB_agent_tool at 0x3197c7610>,
 'get_equity_fundamental_historical_splits': <BB_agent_tool.BB_agent_tool at 0x32a551710>,
 'get_balance_sheet_json': <BB_agent_tool.BB_agent_tool at 0x3293db4d0>,
 'get_cash_flow_json': <BB_agent_tool.BB_agent_tool at 0x3293e8210>}

In [45]:
tool = BB_agent_tool(
    name="get_income_statement_json",
    description="Given a stock symbol, get the latest income statement data for the company in JSON format",
    openapi_path='/api/v1/equity/fundamental/income',
    parameters={
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    example_parameter_values=[{
        "symbol": "NVDA",
    }],
    singular=1
)

agent_query(f"what was the most recent net income for symbol {symbol}", verbose=False)


get_income_statement_json({'symbol': 'MRK'}) -> {"period_ending": "2023-12-31", "fiscal_period": "FY", "fiscal_year": 2023, "filing_date": "2024-04-...


The most recent net income for Merck & Co., Inc. (symbol: MRK) for the fiscal year ending December 31, 2023, was \\$365 million.

For more details, you can refer to their [latest 10-K filing](https://www.sec.gov/Archives/edgar/data/310158/000119312524093194/d807955d10ka.htm).

In [46]:
tool = BB_agent_tool(
    name="get_fundamental_metrics_json",
    description="Given a stock symbol, get fundamental metrics for the company in JSON format.",
    openapi_path='/api/v1/equity/fundamental/metrics',
    parameters={
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    example_parameter_values=[{
        "symbol": "NVDA",
    }],
)

agent_query(f"what was the most recent PE ratio for stock symbol {symbol}", verbose=False)


get_fundamental_metrics_json({'symbol': 'MRK'}) -> [{"symbol": "MRK", "market_cap": 333650000000.0, "pe_ratio": 146.64, "foward_pe": 13.3, "eps": 0.9, ...


The most recent PE ratio for Merck & Co., Inc. (symbol: MRK) is 146.64.

In [47]:
tool = BB_agent_tool(
    name="get_fundamental_ratios_json",
    description="Given a stock symbol, get fundamental valuation ratios for the company in JSON format.",
    openapi_path='/api/v1/equity/fundamental/ratios',
    parameters={
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    example_parameter_values=[{
        "symbol": "NVDA",
    }],
    singular=1
)

agent_query(f"what was the most recent price to sales ratio for stock symbol {symbol}", verbose=False)


get_fundamental_metrics_json({'symbol': 'MRK'}) -> [{"symbol": "MRK", "market_cap": 333650000000.0, "pe_ratio": 146.64, "foward_pe": 13.3, "eps": 0.9, ...


The most recent price to sales ratio for Merck & Co., Inc. (symbol: MRK) is 5.46.

In [48]:
tool = BB_agent_tool(
    name="get_equity_fundamental_multiples",
    description="Given a stock symbol, get fundamental valuation multiples for the company in JSON format.",
    openapi_path='/api/v1/equity/fundamental/multiples',
    parameters={
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    example_parameter_values=[{
        "symbol": "NVDA",
    }],
    singular=1
)

agent_query(f"what was the most recent revenue per share for stock symbol {symbol}", verbose=False)


get_equity_search_symbol({'query': 'MRK', 'limit': 1}) -> [{"symbol": "FDT", "name": "FIRST TR EXCH TRD ALPHDX FD II DEV MRK EX US", "dpm_name": "Belvedere Tr...
get_equity_search_symbol({'query': 'Merck', 'limit': 1}) -> [{"symbol": "MRK", "name": "MERCK & CO INC COM", "dpm_name": "Wolverine Trading, LLC", "post_station...
get_equity_fundamental_multiples({'symbol': 'MRK'}) -> {"symbol": "MRK", "revenue_per_share_ttm": 24.25424397947098, "net_income_per_share_ttm": 0.91038294...


The most recent revenue per share for Merck & Co., Inc. (ticker: MRK) is **\\$24.25**.

In [49]:
# tool = BB_agent_tool(
#     name="get_historical_eps",
#     description="Given a stock symbol, get historical earnings per share data for the company in JSON format.",
#     openapi_path='/api/v1/equity/fundamental/historical_eps',
#     parameters={
#         "symbol": {
#             "type": "string",
#             "description": "The stock symbol."
#         }
#         },
#     example_parameter_values=[{
#         "symbol": "NVDA",
#     }],
# )

# 

In [50]:
# agent_query(f"what was the EPS for {company} in the quarter ended 2022-09-30?", verbose=False)


In [51]:
tool = BB_agent_tool(
    name="get_equity_fundamental_dividend",
    description="Given a stock symbol, get the latest dividend data for the company in JSON format.",
    openapi_path='/api/v1/equity/fundamental/dividends',
    parameters={
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    example_parameter_values=[{
        "symbol": "NVDA",
    }],
)

agent_query(f"what was the latest dividend data for stock symbol {symbol}?", verbose=False)


get_equity_fundamental_dividend({'symbol': 'MRK'}) -> [{"ex_dividend_date": "2024-03-14", "amount": 0.77, "label": "March 14, 24", "adj_dividend": 0.77, "...


The latest dividend data for stock symbol MRK (Merck & Co., Inc.) is as follows:

- **Ex-Dividend Date:** March 14, 2024
- **Amount:** \\$0.77
- **Record Date:** March 15, 2024
- **Payment Date:** April 05, 2024
- **Declaration Date:** January 23, 2024

If you have any further questions or need additional details, feel free to ask!

In [52]:
agent_query(f"what was the dividend data as of the end of 2022 for stock symbol {symbol}?", verbose=False)


get_equity_search_symbol({'query': 'MRK', 'limit': 5}) -> [{"symbol": "FDT", "name": "FIRST TR EXCH TRD ALPHDX FD II DEV MRK EX US", "dpm_name": "Belvedere Tr...
get_equity_search_symbol({'query': 'merck', 'limit': 5}) -> [{"symbol": "MRK", "name": "MERCK & CO INC COM", "dpm_name": "Wolverine Trading, LLC", "post_station...
get_equity_fundamental_dividend({'symbol': 'MRK'}) -> [{"ex_dividend_date": "2024-03-14", "amount": 0.77, "label": "March 14, 24", "adj_dividend": 0.77, "...


Here is the relevant dividend data for Merck & Co Inc. (symbol: MRK) at the end of 2022:

- **Ex-dividend date**: December 14, 2022
- **Dividend amount**: \\$0.73 per share
- **Record date**: December 15, 2022
- **Payment date**: January 9, 2023
- **Declaration date**: November 29, 2022

If you have any more questions or need further assistance, feel free to ask!

In [53]:
tool = BB_agent_tool(
    name="get_trailing_dividend_yield_json",
    description="Given a stock symbol, get the 1 year trailing dividend yield for the company over time in JSON format.",
    openapi_path='/api/v1/equity/fundamental/trailing_dividend_yield',
    parameters={
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    example_parameter_values=[{
        "symbol": "NVDA",
    }],
)

agent_query(f"what was the dividend yield for stock symbol {symbol} as of the end of 2023?", verbose=False)


get_trailing_dividend_yield_json({'symbol': 'MRK'}) -> [{"date": "2023-05-16", "trailing_dividend_yield": 0.0244658856}, {"date": "2023-05-17", "trailing_d...


As of the end of 2023, the trailing dividend yield for Merck & Co., Inc. (symbol: MRK) was approximately 2.82%. Here are the precise yields on the last trading days of December 2023:

- December 27, 2023: 2.7412%
- December 28, 2023: 2.7213%
- December 29, 2023: 2.7151%

These numbers reflect the trailing dividend yield on those specific dates.

In [54]:
tool = BB_agent_tool(
    name="get_price_performance_json",
    description="Given a stock symbol, get price performance data for the stock for different time periods in JSON format.",
    openapi_path='/api/v1/equity/price/performance',
    parameters={
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    example_parameter_values=[{
        "symbol": "NVDA",
    }],
)

agent_query(f"what was the performance for {company} from 1 year ago?", verbose=False)


get_equity_search_symbol({'query': 'Merck', 'limit': 1}) -> [{"symbol": "MRK", "name": "MERCK & CO INC COM", "dpm_name": "Wolverine Trading, LLC", "post_station...
get_price_performance_json({'symbol': 'MRK'}) -> [{"symbol": "MRK", "one_day": 0.0239, "wtd": null, "one_week": 0.0168, "mtd": null, "one_month": 0.0...


The performance of Merck (symbol: MRK) over the past year is as follows:

- **1-Year Performance**: +13.2%
- **Current Price**: \\$131.73
- **Volume**: 7,224,765
- **Average Volume**: 8,210,000

The stock has shown a positive return over the given period.

In [55]:
# this might exceed token context making it unreliable
tool = BB_agent_tool(
    name="get_etf_equity_exposure_json",
    description="Given a stock symbol, get the exposure of ETFs to the stock in JSON format.",
    openapi_path='/api/v1/etf/equity_exposure',
    parameters={
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    example_parameter_values=[{
        "symbol": "NVDA",
    }],
)


In [56]:
pd.DataFrame(json.loads(tool(symbol='MRK'))).sort_values('weight', ascending=False)

,equity_symbol,etf_symbol,shares,weight,market_value
445,MRK,FTXH,11684.0,0.075700,1.519621e+06
13,MRK,XLV,18454434.0,0.062123,2.400184e+09
81,MRK,QDVG.DE,1126933.0,0.061660,1.457012e+08
83,MRK,IUHC.L,0.0,0.061095,1.451633e+08
82,MRK,IUHE.AS,0.0,0.061095,1.451633e+08
...,...,...,...,...,...
681,MRK,GPAL,640.0,NaN,6.631040e+04
694,MRK,GDEF,397.0,NaN,5.170131e+04
703,MRK,EUNU.DE,50000.0,NaN,3.868500e+04
715,MRK,GSEU,121.0,NaN,2.030260e+04


In [57]:
agent_query(f"which ETF has the highest weight in {company} ?", verbose=False)


get_equity_search_symbol({'query': 'Merck', 'limit': 1}) -> [{"symbol": "MRK", "name": "MERCK & CO INC COM", "dpm_name": "Wolverine Trading, LLC", "post_station...
get_etf_equity_exposure_json({'symbol': 'MRK'}) -> [{"equity_symbol": "MRK", "etf_symbol": "VTSAX", "shares": 79663326.0, "weight": 0.0066, "market_val...


The ETF with the highest weight in Merck (ticker: MRK) is the XLV ETF, with a weight of approximately 0.0621.

In [58]:
[(k, t.description) for k, t in BB_agent_tool.agent_registry.items()]

[('get_10k_item1_from_symbol',
  "Given a stock symbol, gets item 1 of the company's latest 10-K annual report filing."),
 ('get_equity_search_symbol',
  'Given a search string, get the stock symbol of the top company whose name best matches the search string.'),
 ('get_company_profile_json',
  'Given a stock symbol, get general background data about the company such as company name, industry, and sector data in JSON format'),
 ('get_equity_shorts_short_interest',
  'Given a stock symbol, get data on short volume and days to cover in JSON format.'),
 ('get_equity_fundamental_historical_splits',
  "Given a stock symbol, get the company's historical stock splits in JSON format."),
 ('get_balance_sheet_json',
  'Given a stock symbol, get the latest balance sheet data with assets and liabilities for the company in JSON format.'),
 ('get_cash_flow_json',
  'Given a stock symbol, get the latest cash flow statement data for the company in JSON format.'),
 ('get_income_statement_json',
  'Give

In [62]:
# for k, t in BB_agent_tool.agent_registry.items():
#     print(t.example_code)

In [60]:
len(" ".join([(t.example_code) for k, t in BB_agent_tool.agent_registry.items()]))

23681